In [2]:
import numpy as np

ModuleNotFoundError: No module named 'numpy'

In [3]:
def repmat(matrix, k1, k2=0):
    n1 = len(matrix)
    n2 = len(matrix[0])
    if k2 == 0:
        k2 = k1
    new_matrix = np.empty((k1 * n1, k2 * n2))
    l1 = k1 * n1
    l2 = k2 * n2
    for i in range(l1):
        for j in range(l2):
            new_matrix[i, j] = matrix[i % n1][j % n2]
    return new_matrix
    

In [ ]:
def conv2d(A, B):
    n1, n2 = len(A), len(A[0])
    m1, m2 = len(B), len(B[0])
    A = np.array(A)
    B = np.array(B)
    C = np.zeros((n1-m1+1, n2-m2+1))
    for i in range(n1-m1+1):
        for j in range(n2-m2+1):
            summ = 0
            for u in range(m1 - 1):
                for v in range(m2 - 1):
                    summ = summ + A[i+u,j+v] * B[u,v]
            C[i,j]] = summ
    return C

In [ ]:
def conv2d(A, B):
    n1, n2 = len(A), len(A[0])
    m1, m2 = len(B), len(B[0])
    A = np.array(A)
    B = np.array(B)
    C = np.zeros((n1-m1+1, n2-m2+1))
    for i in range(n1-m1+1):
        for j in range(n2-m2+1):
            summ = 0
            stroka = f'C[{i},{j}]='
            for u in range(m1):
                for v in range(m2):
                    #print(u,v)
                    summ = summ + A[i+u,j+v] * B[u,v]
                    stroka = stroka + f'{A[i+u,j+v]}*{B[u,v]}+'
            stroka = stroka[:-1]
            stroka = stroka + f'={summ}'
            #print(stroka)
            C[i,j] = summ
    return C

In [4]:
print(repmat([[1]], 8)

SyntaxError: incomplete input (3194977455.py, line 1)

In [ ]:
def fmridesign(frametimes, slicetimes=0, events=[1, 0], S=[], hrf_parameters=[5.4, 5.2, 10.8, 7.35, 0.35], shift=[-4.5, 4.5]):
    n = len(frametimes)
    numslices = len(slicetimes)
    if len(events.shape) > 1:
        numevents = len(events[0])
        eventid = events[0]
        numeventtypes = max(eventid)
        eventime = events[1]
        if len(events[1]) >= 3:
            duration = events[3]
        else:
            duration = np.zeros((numevents, 1))
        if len(events[1]) >= 4:
            height = events[4]
        else:
            height = np.ones((numevents, 1))
        mineventime = min(eventime)
        maxeventime = max(eventime)
    else:
        numeventypes = 0
        mineventime = np.inf
        maxeventime = -np.inf
    if S != []:
        numcolS = len(S[1])
    else:
        numcolS = 0

    dt = 0.02
    startime = min(mineventime, min(frametimes) + min([slicetimes, 0]))
    finishtime = max(maxeventime, max(frametimes) + max([slicetimes 0]))
    numtimes = math.ceil((finishtime - startime)/dt) + 1
    numresposes = numeventypes + numcolS
    responce = np.zeros((numtimes, numresponces))

    if len(events.shape) > 1:
        height = heigth/(1 + (duration==0) * (dt - 1))
        for k in range(numevents):
            tip = eventid(k)
            n1 = math.ceil((eventime[k] - startime)/dt) + 1
            n2 = math.ceil((eventime[k] + duration[k] - startime)/dt) + (duration[k]==0)
            if n2 >= n1:
                responce[n1:n2, tip] = responce[n1:n2, tip] + height[k] * np.ones((n2-n1+1, 1))#Проверить по поводу включения
    #if S != []: Можно дописать, но я не буду

    #if len(hrf_paremeters) == 5:
    #    if len(hfr_paremeters.shape) == 1: Тоже нет смысла дописывать
    if len(hrf_paremeters[0]) == 1:
        hrf_parameters = repmat(hrf_parameters, numresponses, 1)
    if len(shift[0]) == 1:
        shift = repmat(shift, numresponses, 1)

    eventmatrix = np.zeros((numtimes, numresponces, 1))
    nd = 41
    X_cache = {}
    X_cache['W'] = np.zeros((nd, numresponces, 5))

    for k in range(numresponses):
        Delta1 = shift[k, 0]
        Delta2 = shift[k, 1]
        #Тоже не пишем
        peak1 = hrf_parameters[k,0]
        fwhm1 = hrf_parameters[k,1]
        peak2 = hrf_parameters[k,2]
        fwhm2 = hrf_parameters[k,3]
        dip = hrf_parameters[k,4]
        numlags = math.ceil((max(peak1 + 3 * fwhm1 ,peak2 + 3 * fwhm2) + Delta2 - Delta1) / dt) + 1
        numlags = min(numlags, numtimes)
        time = np.array([i*dt for i in range(numlags)])

        tinv = (time > 0) / (time + (time <= 0))#не понятно
        if peak1 > 0 and fwhm1 > 0:
            alpha1 = peak1**2 / fwhm1**2*8*math.log(2)
            beta1 = fwhm1**2 / peak1 / 8 / math.log(2)
            gamma1 = (time/peak1)**alpha1 * math.exp(-(time - peak1) / beta1)
            d_gamma1 = -(alpha1*tinv - 1/beta1) * gamma1
        else:
            gamma1 = min(abs(time - peak1)) == abs(time - peak1)
            d_gamma1 = np.zeros((numlags, 1))

        if peak2 > 0 and fwhm2 > 0:
            alpha2 = peak2**2 / fwhm2**2*8*math.log(2)
            beta1 = fwhm2**2 / peak2 / 8 / math.log(2)
            gamma1 = (time/peak2)**alpha2 * math.exp(-(time - peak2) / beta2)
            d_gamma1 = -(alpha2*tinv - 1/beta2) * gamma2
        else:
            gamma2 = min(abs(time - peak2)) == abs(time - peak2)
            d_gamma2 = np.zeros((numlags, 1))
        hrf = gamma1 - dip * gamma2
        d_hrf = d_gamma1 - dip * d_gamma2
        HS = [hrf, d_hrf]/sum(hrf)
        temp = conv2d(responce[:,k], HS)
        eventmatrix[:, k, 0:1] = temp[:numtimes, :]
        
        H = np.zeros((numlags, nd))
        delta = ([:nd] - 1) / (nd - 1) * (Delta2 - Delta1) + Delta1
        for i in range(nd):
            #
            t = (time + Delta1 - delta(i)) * ((time + Delta1) > delta(i))
            if peak1 > 0 and fwhm1 > 0:
                gamma1 = (t/peak1)**alpha1*math.exp(-(t-peak1)/beta1)
            else:
                gamma1 = min(abs(t-peak1))==abs(t-peak1)
    
            if peak2 > 0 and fwhm2 > 0:
                gamma2 = (t/peak2)**alpha2*math.exp(-(t-peak2)/beta2)
            else:
                gamma1 = min(abs(t-peak2))==abs(t-peak2)
            hrf = gamma1 - dip*gamma2
            H[:, i] = hrf/sum(hrf)
    
        origin = -round(Delta1/dt)
        HS0 = [np.zeros((origin,1)), HS[:(numlags-origin), :]]
        WS = np.linalg.pinv(HS0) * H
        X_cache['W'][:, k, 0:1] = np.transpose(WS)
        prcnt_var_taylor = sum(sum(H * (HS0 * WS)))/sum(sum(H*H))*100
    
        U, SS, V = np.linalg.svd(H)#svd(H,0)
        prcnt_var_spectral = (SS[0,0]**2 + SS[1,1]**2)/sum(np.diag(SS)**2)*100
        sumU = sum(U[:, 0])
        US = U[:, 0:1]/sumU
        WS = V[:, 0:1]*SS[0:1, 0:1] * sumU
        if delta * WS[:, 1] < 0:
            US[:, 1] = -US[:, 1]
            WS[:, 1] = -WS[:, 1]
        temp = conv2d(responce[:, k], US)
        eventmatrix[:, k, 2:3] = temp[[0:numtimes] - round(Delta1/dt), :] 
        X_cache['W'][:, k, 2:3] = WS
        X_cache['W'][:, k, 4] = np.transpose(delta)
    
        if not all(WS[:,0] > 0):
            print('Warning: use only for magnitudes, not delays \n first coef not positive for stimulus', num2str(k))
        cubic_coef = np.linalg.pinv([delta.transpose(), delta.transpose()**3]) * (WS[:, 1]/WS[:, 0])
        if np.prod(cubic_coef) < 0:
            print('\nWarning: use only for magnitudes, not delays \n svd ratio not invertible for stimulus ', num2str(k))
    X_cache['X'] = np.zeros((n, numresponces, 4, numslices))

    for slic in range(numslices):
        subtime = math.ceil((frametimes + slicetimes[slic] - startime)/dt) + 1
        X_cache['X'][:, :, :, slic] = event_matrix[subtime, :, :]
        
    X_cache['TR'] = (max(frametimes) - min(frametimes))/(len(frametimes) - 1)

    return X_cache
    
                
        
        

    